In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import random
from sklearn.neighbors import NearestNeighbors
import numpy as np
class Smote:
    def __init__(self,samples,N=10,k=5):
        self.n_samples,self.n_attrs=samples.shape
        self.N=N
        self.k=k
        self.samples=samples
        self.newindex=0

    def over_sampling(self):
        N=int(self.N/100)
        self.synthetic = np.zeros((self.n_samples * N, self.n_attrs))
        neighbors=NearestNeighbors(n_neighbors=self.k).fit(self.samples)  
#         print ('neighbors',neighbors)
        for i in range(len(self.samples)):
#             print('samples',self.samples[i])
            nnarray=neighbors.kneighbors(self.samples[i].reshape((1,-1)),return_distance=False)[0]  #Finds the K-neighbors of a point.
#             print ('nna',nnarray)
            self._populate(N,i,nnarray)
        return self.synthetic


    # for each minority class sample i ,choose N of the k nearest neighbors and generate N synthetic samples.
    def _populate(self,N,i,nnarray):
        for j in range(N):
#             print('j',j)
            nn=random.randint(0,self.k-1)  #包括end
            dif=self.samples[nnarray[nn]]-self.samples[i]
            gap=random.random()
            self.synthetic[self.newindex]=self.samples[i]+gap*dif
            self.newindex+=1
#             print(self.synthetic)
            
# a=np.array([[1,2,3],[4,5,6],[2,3,1],[2,1,2],[2,3,4],[2,3,4]])
# s=Smote(a,N=200)
# n= s.over_sampling()
# print(np.shape(n))

In [3]:

def save_balanced_data( train_i,train_t,test_i,test_t):
    fp = open('balanced_train_v1.txt','w+')
    for i in range(len(train_i)):
        d_str = str(train_t[i]) + '\t' +str(train_i[i,:])[1:-1] +'\n'
        fp.write(d_str)
    fp.close()

    fp = open('testset.txt_v1', 'w+')
    for i in range(len(test_i)):
        d_str = str(test_t[i]) + '\t' + str(test_i[i, :])[1:-1] + '\n'
        fp.write(d_str)
    fp.close()

    pass


def balance_data(aep_i,aep_t, non_aep_i,non_aep_t,method ='downsampling', test_aep_sample= 30,test_nonaep_sample= 1200):
    """

    :param aep_i:
    :param aep_t:
    :param non_aep_i:
    :param non_aep_t:
    :param method:
    :param test_aep_sample:
    :param test_nonaep_sample:
    :return:
    """

    # split test set and train set
    test_aep_sample = int(0.3*len(aep_i))
    
    aep_test_i = aep_i[0:test_aep_sample,:]
    aep_test_t = aep_t[0:test_aep_sample]
    test_nonaep_sample = int(0.3*len(non_aep_i))
    non_aep_test_i = non_aep_i[0:test_nonaep_sample,:]
    non_aep_test_t = non_aep_t[0:test_nonaep_sample]

    # test set
    test_i = np.concatenate((aep_test_i,non_aep_test_i),axis=0)
    test_t = []
    test_t.extend(aep_test_t)
    test_t.extend( non_aep_test_t)

    # train set
    aep_i =aep_i[test_aep_sample:,:]
    aep_t = np.array(aep_t[test_aep_sample:])
    non_aep_i = non_aep_i[test_nonaep_sample:, :]
    non_aep_t = non_aep_t[test_nonaep_sample:]

 
    # balance training set here
    if method == 'oversampling':
        sample_range = len(aep_i)
        indices = np.random.randint(sample_range, size=len(non_aep_i))
        new_aep_i = aep_i[indices,:]
        new_aep_t = aep_t[indices]
        
        train_i = np.concatenate((new_aep_i, non_aep_i),axis=0)
        train_t =[]
        train_t.extend(new_aep_t)
        train_t.extend(non_aep_t)


    elif method == 'downsampling':
        sample_range = len(non_aep_i)
        indices = np.random.randint(sample_range, size=len(aep_i))
        new_nonaep_i = non_aep_i[indices,:]
        new_nonaep_t = non_aep_t[indices]
        
        train_i = np.concatenate((aep_i,new_nonaep_i),axis=0)
        train_t =[]
        train_t.extend(aep_t)
        train_t.extend(new_nonaep_t)
        pass
    elif method == 'SMOTE':
        
        s= Smote(aep_i,N=10*int(5000/len(aep_i)))
        new_aep_i = s.over_sampling()
        new_aep_t = [1.0]*len(new_aep_i)
        indices = np.random.randint(len(non_aep_i), size=len(new_aep_i))
        new_nonaep_i = non_aep_i[indices,:]
        new_nonaep_t = non_aep_t[indices]
        
        train_i = np.concatenate((new_aep_i,new_nonaep_i),axis=0)
        train_t =[]
        train_t.extend(new_aep_t)
        train_t.extend(new_nonaep_t)
    else:
        sample_size = 500
        indices = np.random.randint(len(non_aep_i), size=sample_size)
        new_nonaep_i = non_aep_i[indices,:]
        new_nonaep_t = non_aep_t[indices]
        indices = np.random.randint(len(aep_i), size=sample_size)
        new_aep_i = aep_i[indices,:]
        new_aep_t = aep_t[indices]
        
        train_i = np.concatenate((new_aep_i,new_nonaep_i),axis=0)
        train_t =[]
        train_t.extend(new_aep_t)
        train_t.extend(new_nonaep_t)
        pass


    return train_i,train_t,test_i,test_t



def load_balanced_data():
    fp = open('db4_Diff1f_100_128w_AEP2_gp2_ver1-1_ece856.txt', 'r+')
    lines = fp.readlines()
    aep_i = []
    aep_t = []
    non_aep_i = []
    non_aep_t = []

    for line in lines:
        line = line.split('\n')[0]
        data = [float(s) for s in line.split(' ') if s != '']
        aep_i.append(data)
        aep_t.append(1)
    fp.close()

    aep_i = np.array(aep_i)
    aep_t = np.array(aep_t)

    fp = open('db4_Diff1f_100_128w_nonAEP2_gp2_ver1-1_ece856.txt', 'r+')
    lines = fp.readlines()
    for line in lines:
        line = line.split('\n')[0]
        data = [float(s) for s in line.split(' ') if s != '']
        non_aep_i.append(data)
        non_aep_t.append(0)
    fp.close()
    non_aep_i = np.array(non_aep_i)
    non_aep_t = np.array(non_aep_t)

#     train_i, train_t, test_i, test_t= balance_data(aep_i,aep_t,non_aep_i,non_aep_t,method='downsampling')
    train_i, train_t, test_i, test_t= balance_data(aep_i,aep_t,non_aep_i,non_aep_t,method='SMOTE')
    save_balanced_data(train_i, train_t, test_i, test_t)
    train_i, train_t, test_i, test_t = np.array(train_i),np.array(train_t),np.array(test_i),np.array(test_t)
    return train_i, train_t, test_i, test_t


In [9]:
class MLFF():
    def __init__(self,input_shape = 27, bias=False):
#         np.random.seed(1)
        self.weights = {}
        self.bias_enabled = bias
        self.weight_correction = {}
        self.delta ={}
        self.input_shape = input_shape
        self.num_layer = 1 # initialize input layer
        self.func_pair = {'logistic':(self.logistic,self.d_logistic),'relu':(self.relu,self.d_relu),'tanh':(self.tanh,self.d_tanh)}
        self.activate = {}
        self.derivative = {}
        
    def add_layer(self,shape, activation=None,initializer='random'):
        if initializer =='zero':
            # zero initializer
            self.weights[self.num_layer] = np.vstack((np.zeros(shape[1]), np.zeros((shape))))
        else:
            #random initializer
            # The first row is bias 
#             np.random.seed(2)
            self.weights[self.num_layer] = np.vstack((10*np.random.random(shape[1])-5, 10*np.random.random(shape)-5))
        self.weight_correction[self.num_layer]= np.zeros((shape[0]+1, shape[1]))
        
        self.num_layer += 1
        
        # the address to activation function
        if activation is not None and activation is not '':
            self.activate[self.num_layer] = self.func_pair[activation][0]
            self.derivative[self.num_layer] = self.func_pair[activation][1]
        else:
            self.activate[self.num_layer] = self.logistic
            self.derivative[self.num_layer] = self.d_logistic
        
    def logistic(self, net):
        k = 0.01
#         return np.exp(net)/(1+ k*np.exp(net))
        return 1/(1+ np.exp(-k*net))
    
    def d_logistic(self,net,out):
        """
        out: numpy array
        """
        k = 0.01
#         return k*np.exp(-k*net)/np.square(1+ np.exp(-k*net))
        return k*out*(1.0- out)

    
    def tanh(self,net):
        return np.tanh(net)
    
    def d_tanh(self,net,o):
        return 1.0-np.square(o)
    
    def relu(self,net):
        k =1.0
        k1= 0.5
        ma = k*np.maximum(net, np.zeros(np.shape(net)))
        mi = k1*np.minimum(net, np.zeros(np.shape(net)))
        return ma+mi
    
    def d_relu(self, net,o):
        k = 1.0
        k1 = 0.5
        z = np.zeros(np.shape(net))
        m = np.maximum(z,net)
        deriv = np.where(m>0.0,k,m)
        deriv = np.where(deriv==0.0,k1,deriv)
        return deriv
    
    
    def feedforward(self,input,bias= False):
        """
        input: 1-D array
        """
        out = {}
        out[1] = input
        for i in range(self.num_layer-1):
            layer = i+1
            net = np.dot(out[layer], self.weights[layer][1:,:])
            if bias:
                net += self.weights[layer][0,:]
            out[layer+1]= self.activate[layer+1](net)
            
        return out, out[self.num_layer]
    
        
    def backforward(self,out, target,bias = False):
        """
        Apply General delta rule to backforward
        out: the dictionary storing all outputs in different layers
        """
        pred = out[self.num_layer]
        dE_o = -(target- pred)
        prev_net = np.dot(out[self.num_layer-1], self.weights[self.num_layer-1][1:,:])
        if bias:
            prev_net += self.weights[self.num_layer-1][0,:]
        
#         print("predict:",out,'target:',target,"output-net:",prev_net)
        
        
        self.delta[self.num_layer] = -np.multiply(dE_o, self.derivative[self.num_layer](prev_net,pred) )
        # calculate delta of each layer
        # calculate delta from layer-2 to the last second layer,since the input layer doesn't have delta
        # delta  of the output layer is determined by output
        for layer in reversed(range(2, self.num_layer)):
            last_delta = self.delta[layer+1] 
            
            dE_o =np.dot(self.weights[layer][1:,:], last_delta)
            prev_net = np.dot(out[layer-1], self.weights[layer-1][1:,:])
            if bias:
                prev_net += self.weights[layer-1][0,:]
                
            d_f =self.derivative[layer](prev_net,out[layer])
            self.delta[layer] = -np.multiply(dE_o, d_f)
            
#             print('last-delta:',np.shape(last_delta),last_delta, "weights-",layer,': ',self.weights[layer][1:,:])
#             print('layer:',layer)
#             print('prev_net:',prev_net)
#             print('out:',out[layer])
#             print('dE_o-',layer,": ",dE_o,'d_f:',d_f,'delta:',np.shape(self.delta[layer]),self.delta[layer])
        
        #calculate weight correction
        for layer in range(1, self.num_layer):
            self.weight_correction[layer][1:,:] += np.multiply(self.delta[layer+1],np.reshape(out[layer], (len(out[layer]), 1)))
            #update bias if enabled
            if bias:
                self.weight_correction[layer][0,:] += 1e-3*self.delta[layer+1]
            
        return None
    
    
    def predict(self, inputs,bias=False):
        _, preds = self.feedforward(inputs,bias)
        preds = np.array([1 if p>=0.5 else 0 for p in preds])
        return preds
    
    def gradient_descent(self,alpha,beta,momentum= None):
        
        new_momentum= self.weight_correction.copy()
        for layer in range(1, self.num_layer):
            if momentum is not None:
                self.weight_correction[layer] = alpha*self.weight_correction[layer] + beta* momentum[layer]
            else:
                self.weight_correction[layer] = alpha*self.weight_correction[layer]
            self.weights[layer] += self.weight_correction[layer]
            #update momentum
            new_momentum[layer] = self.weight_correction[layer]
            #reset weight correction after updating weights
            self.weight_correction[layer]= np.zeros(np.shape(self.weight_correction[layer]))

            
#             print("weight corrections:", self.weight_correction[layer])
#             print("weights:",self.weights)`
        return new_momentum
        
    def train(self,inputs,targets,alpha0=0.98, beta0=0.98, bias =False,momentum_enabled = False,max_iteration = 1000):
        
        momentum  = None
        Etss = []
        sensitivity = []
        specification = []
        
        for epoch in range(max_iteration):
            err = 0.0
            #need to decrease learning rate alpha, otherwise, it would not decrease error, error doesn't converge
#             alpha = alpha0/float(0.1*epoch+1)
            alpha = 0.3
            beta = beta0/float(0.1*epoch+1)
            
#             alpha = alpha0*float(max_iteration-epoch)/max_iteration
#             beta = beta0* float(max_iteration-epoch)/max_iteration
            
            for i in range(len(inputs)):
                outs, pred= self.feedforward(inputs[i,:],bias)
#                 print('pred:',pred)
                err  += 0.5*np.square(targets[i]-pred)
                self.backforward(outs,targets[i],bias)
        
                m = self.gradient_descent(alpha, beta,momentum)
                if momentum:
                    momentum = m
#             err /= len(inputs)
            
            if epoch % 100 == 0:
                preds = self.predict(inputs,bias)
                eap_cnts = float(np.sum(targets))
                TP = np.sum(preds[np.where(targets == 1)])
                sens = TP/eap_cnts
                
                noneap_cnts =float(len(targets) - float(np.sum(targets))) 
                TN = len(preds[np.where(targets == 0)])- np.sum(preds[np.where(targets == 0)])
                spec = TN/noneap_cnts
                sensitivity.append(sens)
                
                specification.append(spec)
                Etss.append(err)
                
                print("Epoch:", epoch,"Error:",err,"Sensitivity:",sens,eap_cnts,"Specification:",spec,noneap_cnts)
                
                
        return Etss, specification, sensitivity
            
            
        
        
        
        
        
    def Kfold_CV(self,inputs,targets,k=5,alpha0=0.95, beta0=0.98, bias =False,momentum_enabled = False,max_iteration = 1000):
        
        momentum  = None
        Etss = []
        sensitivity = []
        specification = []
        CV_Etss = []
        
        input_num = len(inputs)
        indices = [i for i in range(input_num)]
        partition_size = int(float(input_num)/k)
        partitions = []
        partitions_t = []
        for i in range(k-1):
            ind = np.array([indices.pop(np.random.choice(np.arange(len(indices)), size=1)[0]) for j in range(partition_size) ])
            p = inputs[ind,:]
            p_t=  targets[ind]
            partitions.append(p)
            partitions_t.append(p_t)
        ind = np.array(indices)
        partitions.append(inputs[ind,:])
        partitions_t.append(targets[ind])
        
        
        train_inputs = []
        train_targets= []
        valid_inputs = []
        valid_targets = []
        if k ==1:
            valid_inputs.append(partitions[0])
            valid_targets.append(partitions_t[0])
            train_inputs.append(partitions[0]) 
            train_targets.append(partitions_t[0]) 
        else:
            for i in range(k):
                p= partitions.copy()
                p_t=partitions_t.copy()
                valid_inputs.append(p.pop(i))
                valid_targets.append(p_t.pop(i))
                train_inputs.append(np.concatenate(p) ) 
                train_targets.append(np.concatenate(p_t)) 

            del partitions
            del partitions_t

        for epoch in range(max_iteration):
            err = 0.0
            #need to decrease learning rate alpha, otherwise, it would not decrease error, error doesn't converge
            alpha = alpha0/float(epoch+1)
            beta = beta0/float(epoch+1)

            cv_err = 0.0
            #loop through each fold
            for i in range(len(train_inputs)):
                train_set = train_inputs[i]
                train_t = train_targets[i]
                valid_set = valid_inputs[i]
                valid_t = valid_targets[i]
#                 print("train target:",train_targets)
                #loop through each sample
                train_err = 0.0
                for j in range(len(train_set)):
                    outs, pred= self.feedforward(train_set[j,:],bias)
    #                 print('pred:',pred)
                    train_err  += 0.5*np.square(train_t[j]-pred)
                    self.backforward(outs,train_t[j],bias)
                #batch training 
                m = self.gradient_descent(alpha, beta,momentum)
                if momentum:
                    momentum = m
                train_err /= len(train_set)
                
                #validate 
                preds = self.predict(valid_set,bias)
                #calculate MSE 
                mse = 0.5*np.sum(np.square(valid_t-preds))/len(preds)
                cv_err += mse
                err += train_err
                
            cv_err /= k
            err /= k
            if epoch % 100 == 0:
                preds = self.predict(inputs,bias)
                eap_cnts = float(np.sum(targets))
#                 p1= preds[np.where(targets > 0)]
                TP = np.sum(preds[np.where(targets == 1)])
                sens = TP/eap_cnts
#                 p2=preds[np.where(targets == 0)]
                
                noneap_cnts =float(len(targets) - float(np.sum(targets))) 
                TN = len(preds[np.where(targets == 0)])- np.sum(preds[np.where(targets == 0)])
                spec = TN/noneap_cnts
                sensitivity.append(sens)
                
                specification.append(spec)
                Etss.append(err)
                CV_Etss.append(cv_err)

                print("CV_Err:",cv_err,"Error:",err,"Sensitivity:",sens,eap_cnts,"Specification:",spec,noneap_cnts)
                
                
        return Etss, specification, sensitivity
            
            
        
        
        

In [10]:
train_i, train_t, test_i, test_t = load_balanced_data()
from sklearn.utils import shuffle
train_i, train_t = shuffle(train_i, train_t)
print("shape:",np.shape(train_i),np.shape(train_t),np.shape(test_i))
# print(train_t[200:])

shape: (944, 27) (944,) (768, 27)


In [11]:
norm_train_i = (train_i- np.min(train_i,axis=0))/(np.max(train_i,axis=0) - np.min(train_i,axis=0))    
test_i = (test_i- np.min(test_i,axis=0))/(np.max(test_i,axis=0) - np.min(test_i,axis=0))    

In [17]:
d = 27
model_1 = MLFF(input_shape  = d)
model_1.add_layer((d,2*d+1),activation='relu')
model_1.add_layer((2*d+1,1),activation='')
weight_copy_1 = model_1.weights.copy()
Etss_1, specification_1, sensitivity_1 = model_1.train(norm_train_i,train_t,alpha0=0.001,
                                                       bias=False,momentum_enabled=False ,max_iteration = 4000)

Epoch: 0 Error: [ 119.41591667] Sensitivity: 0.190677966102 472.0 Specification: 0.73093220339 472.0
Epoch: 100 Error: [ 126.10980896] Sensitivity: 0.118644067797 472.0 Specification: 0.423728813559 472.0
Epoch: 200 Error: [ 117.67306972] Sensitivity: 0.309322033898 472.0 Specification: 0.728813559322 472.0
Epoch: 300 Error: [ 121.20052793] Sensitivity: 0.205508474576 472.0 Specification: 0.654661016949 472.0
Epoch: 400 Error: [ 118.02582065] Sensitivity: 0.277542372881 472.0 Specification: 0.697033898305 472.0
Epoch: 500 Error: [ 117.2893024] Sensitivity: 0.110169491525 472.0 Specification: 0.796610169492 472.0
Epoch: 600 Error: [ 114.89610198] Sensitivity: 0.440677966102 472.0 Specification: 0.735169491525 472.0
Epoch: 700 Error: [ 116.9474078] Sensitivity: 0.398305084746 472.0 Specification: 0.720338983051 472.0
Epoch: 800 Error: [ 115.58584441] Sensitivity: 0.491525423729 472.0 Specification: 0.692796610169 472.0
Epoch: 900 Error: [ 114.58450063] Sensitivity: 0.453389830508 472.0 S

In [16]:
preds = model_1.predict(test_i,bias=False)
acc = float(list(np.logical_xor(test_t,preds)).count(False))/len(test_t)
print("Model 1 acc:",acc)

Model 1 acc: 0.96484375


In [42]:
model_2 = MLFF(input_shape  = d)
model_2.add_layer((d,2*d+1),activation='tanh')
model_2.add_layer((2*d+1,1),activation='')
model_2.weights = weight_copy_1.copy()
weight_copy_2 = model_2.weights.copy()
Etss_2, specification_2, sensitivity_2 = model_2.train(norm_train_i,train_t,
                                                       bias=False,momentum_enabled=False ,max_iteration = 10000)

Epoch: 0 Error: [ 117.8694753] Sensitivity: 0.779661016949 472.0 Specification: 0.266949152542 472.0
Epoch: 500 Error: [ 117.52613283] Sensitivity: 0.891949152542 472.0 Specification: 0.186440677966 472.0
Epoch: 1000 Error: [ 117.76480204] Sensitivity: 0.300847457627 472.0 Specification: 0.728813559322 472.0
Epoch: 1500 Error: [ 117.72975482] Sensitivity: 0.493644067797 472.0 Specification: 0.622881355932 472.0
Epoch: 2000 Error: [ 117.74881725] Sensitivity: 0.415254237288 472.0 Specification: 0.716101694915 472.0
Epoch: 2500 Error: [ 117.76820609] Sensitivity: 0.402542372881 472.0 Specification: 0.720338983051 472.0
Epoch: 3000 Error: [ 117.7745412] Sensitivity: 0.419491525424 472.0 Specification: 0.694915254237 472.0
Epoch: 3500 Error: [ 117.77629725] Sensitivity: 0.421610169492 472.0 Specification: 0.661016949153 472.0
Epoch: 4000 Error: [ 117.77639893] Sensitivity: 0.421610169492 472.0 Specification: 0.658898305085 472.0
Epoch: 4500 Error: [ 117.77585244] Sensitivity: 0.42372881355

In [43]:
preds = model_2.predict(test_i,bias=True)
acc = float(list(np.logical_xor(test_t,preds)).count(False))/len(test_t)
print("Model 2 acc:",acc)


Model 2 acc: 0.4466145833333333


In [ ]:
model_2_b = MLFF(input_shape  = d)
model_2_b.add_layer((d,2*d+1),activation='tanh')
model_2_b.add_layer((2*d+1,1),activation='')
model_2_b.weights = weight_copy_2.copy()
Etss_2_b, specification_2_b, sensitivity_2_b = model_2_b.train(norm_train_i,train_t,
                                                       bias=True,momentum_enabled=False ,max_iteration = 10000)


Epoch: 0 Error: [ 117.35719011] Sensitivity: 0.516949152542 472.0 Specification: 0.582627118644 472.0
Epoch: 500 Error: [ 117.87183025] Sensitivity: 0.502118644068 472.0 Specification: 0.576271186441 472.0
Epoch: 1000 Error: [ 117.95443143] Sensitivity: 0.258474576271 472.0 Specification: 0.735169491525 472.0
Epoch: 1500 Error: [ 117.97887795] Sensitivity: 0.222457627119 472.0 Specification: 0.771186440678 472.0
Epoch: 2000 Error: [ 117.98808431] Sensitivity: 0.213983050847 472.0 Specification: 0.786016949153 472.0


In [ ]:
preds = model_2_b.predict(test_i,bias=True)
acc = float(list(np.logical_xor(test_t,preds)).count(False))/len(test_t)
print("Model 2 acc:",acc)


In [ ]:
model_3 = MLFF(input_shape  = d)
model_3.add_layer((d,2*d+1),activation='tanh')
model_3.add_layer((2*d+1,1),activation='')
model_3.weights = weight_copy_1.copy()
Etss_3, specification_3, sensitivity_3 = model_3.train(norm_train_i,train_t,bias=False,momentum_enabled=True ,max_iteration = 10000)

In [19]:
preds = model_3.predict(test_i,bias=False)
acc = float(list(np.logical_xor(test_t,preds)).count(False))/len(test_t)
print('Model 3 acc:',acc)

Model 3 acc: 0.9427083333333334


In [20]:
model_4 = MLFF(input_shape  = d)
model_4.add_layer((d,2*d+1),activation='relu')
model_4.add_layer((2*d+1,1),activation='')
model_4.weights = weight_copy_2.copy()
Etss_4, specification_4, sensitivity_4 = model_4.train(norm_train_i,train_t,bias=True,momentum_enabled=True ,max_iteration = 10000)

Epoch: 0 Error: [ 117.3133655] Sensitivity: 0.993644067797 472.0 Specification: 0.0444915254237 472.0
Epoch: 500 Error: [ 118.55577573] Sensitivity: 0.444915254237 472.0 Specification: 0.474576271186 472.0
Epoch: 1000 Error: [ 117.34441412] Sensitivity: 0.622881355932 472.0 Specification: 0.533898305085 472.0
Epoch: 1500 Error: [ 117.70982959] Sensitivity: 0.593220338983 472.0 Specification: 0.485169491525 472.0
Epoch: 2000 Error: [ 118.27212092] Sensitivity: 0.597457627119 472.0 Specification: 0.432203389831 472.0
Epoch: 2500 Error: [ 118.62136849] Sensitivity: 0.588983050847 472.0 Specification: 0.417372881356 472.0
Epoch: 3000 Error: [ 118.68218042] Sensitivity: 0.610169491525 472.0 Specification: 0.385593220339 472.0
Epoch: 3500 Error: [ 118.58656553] Sensitivity: 0.622881355932 472.0 Specification: 0.379237288136 472.0
Epoch: 4000 Error: [ 118.42660592] Sensitivity: 0.64406779661 472.0 Specification: 0.366525423729 472.0
Epoch: 4500 Error: [ 118.23774985] Sensitivity: 0.6483050847

In [21]:
preds = model_4.predict(test_i,bias=True)
acc = float(list(np.logical_xor(test_t,preds)).count(False))/len(test_t)
print("Model 4 acc:",acc)

Model 4 acc: 0.4205729166666667


In [ ]:
# save weights
def save_weights(w,i):
    import json
    fp = open("weights/model_case"+str(i)+".txt","w+")
    fp.write(json.dumps(w))
    fp.close()

d1= {'w1':weight_copy_1[1].tolist(),'w2':weight_copy_1[2].tolist()}
d2= {'w1':weight_copy_2[1].tolist(),'w2':weight_copy_2[2].tolist()}
save_weights( d1,1)
save_weights( d2,2)

In [ ]:
#save error sensitivity 
# Etss_1 = [i[0] for i in Etss_1]
# Etss_2 = [i[0] for i in Etss_2]
# Etss_3 = [i[0] for i in Etss_3]
# Etss_4 = [i[0] for i in Etss_4]
dic_Err = {'1':Etss_1,'2':Etss_2,'3':Etss_3,'4':Etss_4}
dic_Sen={'1':sensitivity_1.tolist(),'2':sensitivity_2.tolist(),'3':sensitivity_3.tolist(),'4':sensitivity_4.tolist()}
dic_Spec={'1':specification_1.tolist(),'2':specification_2.tolist(),'3':specification_3.tolist(),'4':specification_4.tolist()}
def save_curves(file_name,d):
    import json
    fp = open(file_name+"_v1.txt","w+")
    fp.write(json.dumps(d))
    fp.close()

save_curves("Etss",dic_Err)
save_curves("Sensitivity",dic_Sen)
save_curves("Specification",dic_Spec)


In [ ]:
#plot data
epochs = [i for i in range(int(10000/100))]
plt.plot(epochs,Etss_1,'r',label='case-1')
plt.plot(epochs,Etss_2,'g',label='case-2')
plt.plot(epochs,Etss_3,'b',label='case-3')
plt.plot(epochs,Etss_4,'y',label='case-4',)
plt.legend(["case-1","case-2","case-3","case-4"])
plt.xlabel("epochs (per 100 epochs)")
plt.ylabel("Sum square error")

In [ ]:
ind= np.argsort(specification_1)
specification_1 = np.array(specification_1)[ind] 
sensitivity_1= np.array(sensitivity_1)[ind]

ind= np.argsort(specification_2)
specification_2 = np.array(specification_2)[ind] 
sensitivity_2= np.array(sensitivity_2)[ind]

ind= np.argsort(specification_3)
specification_3 = np.array(specification_3)[ind] 
sensitivity_3= np.array(sensitivity_3)[ind]

ind= np.argsort(specification_4)
specification_4 = np.array(specification_4)[ind] 
sensitivity_4= np.array(sensitivity_4)[ind]

plt.plot(1-specification_1,sensitivity_1,'r',label='case-1')
plt.plot(1-specification_2,sensitivity_2,'g',label='case-2')
plt.plot(1-specification_3,sensitivity_3,'b',label='case-3')
plt.plot(1-specification_4,sensitivity_4,'y',label='case-4')
plt.legend(["case-1","case-2","case-3","case-4"])
plt.xlabel("False positive")
plt.ylabel("Sensitivity")